In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ml-tp4/sample_submission.csv
/kaggle/input/2021-ml-tp4/class_info.csv
/kaggle/input/2021-ml-tp4/train_label.csv
/kaggle/input/2021-ml-tp4/test/0261.avi
/kaggle/input/2021-ml-tp4/test/0435.avi
/kaggle/input/2021-ml-tp4/test/0309.avi
/kaggle/input/2021-ml-tp4/test/0039.avi
/kaggle/input/2021-ml-tp4/test/0162.avi
/kaggle/input/2021-ml-tp4/test/0402.avi
/kaggle/input/2021-ml-tp4/test/0255.avi
/kaggle/input/2021-ml-tp4/test/0416.avi
/kaggle/input/2021-ml-tp4/test/0354.avi
/kaggle/input/2021-ml-tp4/test/0486.avi
/kaggle/input/2021-ml-tp4/test/0113.avi
/kaggle/input/2021-ml-tp4/test/0106.avi
/kaggle/input/2021-ml-tp4/test/0291.avi
/kaggle/input/2021-ml-tp4/test/0407.avi
/kaggle/input/2021-ml-tp4/test/0247.avi
/kaggle/input/2021-ml-tp4/test/0442.avi
/kaggle/input/2021-ml-tp4/test/0193.avi
/kaggle/input/2021-ml-tp4/test/0044.avi
/kaggle/input/2021-ml-tp4/test/0136.avi
/kaggle/input/2021-ml-tp4/test/0332.avi
/kaggle/input/2021-ml-tp4/test/0311.avi
/kaggle/input/2021-ml-tp4/tes

In [2]:
import os
import numpy as np
import pandas as pd
import tqdm
import time
import cv2
import pickle
import torch

from sklearn.cluster import KMeans
from sklearn.cluster import MiniBatchKMeans
from sklearn.svm import SVC
from scipy.stats import mode
from sklearn.decomposition import PCA


In [3]:
# 베이스라인 달성을 위한 파라미터 제공
arg_img_size = (128, 128)
arg_dense_sift = True
args_local_cluster = 200
args_global_cluster = 200
num_frame = 5
# args_aggr = "bow" # or "vlad" 
args_aggr = "vlad" # baseline2를 위해 "bow" -> "vlad" 로 변경
pca_vlad = 128

# 비디오 전처리 및 프레임 별 특징점(visual word) 추출 (Empty Module 1)

In [4]:
# train 비디오의 행동 분류 label read
root = "/kaggle/input/2021-ml-tp4/"
train_csv = os.path.join(root, "train_label.csv")
train_csv = pd.read_csv(train_csv)
train_csv_arr = np.asarray(train_csv)

# 데이터 셋에 존재하는 행동 분류 정보 read
classinfo = os.path.join(root, "class_info.csv")
classinfo = pd.read_csv(classinfo)
classinfo_arr = np.asarray(classinfo)


train_path = os.path.join(root, "train")
test_path = os.path.join(root, "test")

# train 비디오 경로
train_list = os.listdir(train_path)
train_list.sort()
train_list = [os.path.join(train_path, i) for i in train_list]

# test 비디오 경로
test_list = os.listdir(test_path)
test_list.sort()
test_list = [os.path.join(test_path, i) for i in test_list]

In [5]:
def video_to_frame(video_path, size, num_frame):
    
    #########################################################
    ## 비디오에서 프레임을 추출해주는 함수
    ## 
    ## Input 
    ##     video_path : 한 비디오의 경로
    ##     size : 비디오 내의 프레임을 읽을 때, 원하는 해상도 크기
    ##     num_frames : 한 비디오 내에서 읽을 프레임의 수
    ##
    ## Output
    ##     frames : 읽고 저장한 총 프레임
    #########################################################
    
    cap = cv2.VideoCapture(video_path)
    
    # 한 비디오의 총 프레임 수 반환 및 원하는 프레임 수 많큼 읽기 위해 읽을 프레임의 인덱스 설정
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    sel_ind = np.linspace(0, total_frame-1, num_frame).astype("int")
    
    
    num=0
    frames = []
    for i in range(total_frame):
        
        # 읽을 프레임 인덱스의 경우 프레임 읽어 메모리에 저장, 아닐 경우 지나감
        if i in sel_ind:
            res, frame = cap.read()
            # 원하는 해상도로 조절 및 grayscale로 변환
            frame = cv2.resize(frame, size, interpolation = cv2.INTER_CUBIC)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
            frames.append(frame)
        else:
            res = cap.grab()        
    cap.release()
    frames = np.asarray(frames)

    return frames

In [6]:
def computeSIFT(data, dense=False):
    
    #########################################################
    ## 비디오 내의 프레임에서 특징점(visual word)을 SIFT or DenseSIFT로 추출해주는 함수
    ## 
    ## Input 
    ##     data : 한 비디오에서 읽고 저장한 프레임
    ##     dense : SIFT or DenseSIFT 사용 여부
    ##
    ## Output
    ##     x : 프레임에 대해 추출된 특징점(visual word), dict 형태 -> x[0]이면 0번째 인덱스 프레임의 특징점(visual word) [n,128] 확인 가능
    #########################################################
    
    x = {}
    for i in range(0, len(data)):
        if dense:
            img = data[i]
            step_size = 8
            kp = [cv2.KeyPoint(x, y, step_size) for x in range(0, img.shape[0], step_size) for y in range(0, img.shape[1], step_size)]
            # 영상에서 물체를 추적하거나 인식할 때, 영상과 영상을 매칭할 때 가장 일반적인 방법은 영상에서 주요 특징점(keypoint)을 뽑아서 매칭하는 것입니다. 
            # 특징점을 영어로는 보통 keypoint 또는 interesting point라 부릅니다.
            
            ####### Empty Module 1 : DenseSIFT ########
            # 기본 SIFT 와 동일하게 정의 
            # 기본 SIFT 에서는 detectAndCompute를 사용했지만 
            # Dense SIFT는 위에서 생성한 keypoint를 사용해 compute 만을 진행 
            ###########################################
            
            sift = cv2.SIFT_create() # SIFT 객체 생성
            kp, desc = sift.compute(img, kp) # Dense SIFT : 위에서 생성한 Keypoint를 활용해서 compute 진행
            
            # .compute : keypoint를 이미 찾은 경우, 찾은 keypoint로 부터 descriptor들을 계산하는 함수
            
        else:
            sift = cv2.SIFT_create() # SIFT 객체 생성
            img = data[i]
            kp, desc = sift.detectAndCompute(img, None)
            
            # .detectAndCompute : keypoint를 찾지 않은 경우, keypoint와 descriptor 둘 다 찾아주는 함수
            
        x.update({i : desc})

    return x

In [7]:
# train 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 train_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
train_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(train_list, desc="Extract {} in train data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    train_local_desc.update({vid_path : local_desc})

# test 비디오에서 프레임 추출 및 특징점(visual word) 추출, dict 형태로 test_local_desc[비디오 경로]이면 해당하는 비디오에서 추출한 모든 특징점(visual word) 확인 가능
test_local_desc = {}
for vi, vid_path in enumerate(tqdm.tqdm(test_list, desc="Extract {} in test data".format("dsift" if arg_dense_sift else "sift"))):
    curr_frame = video_to_frame(vid_path, arg_img_size, num_frame)
    local_desc = computeSIFT(curr_frame, arg_dense_sift)
    test_local_desc.update({vid_path : local_desc})


Extract dsift in test data: 100%|██████████| 505/505 [00:31<00:00, 15.92it/s]


In [8]:
print("\n\nAggregate SIFT descriptor")
start = time.time()


# train 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 train_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 train_local_info에서 확인 가능
train_frame_total = []
train_local_desc_total = []
train_local_info = {}
for k, v in train_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            train_local_desc_total.extend(vv)
            train_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        train_local_info.update({k+", "+str(kk) : l_num})
train_local_desc_total = np.asarray(train_local_desc_total)
train_frame_total = np.asarray(train_frame_total)


# test 비디오 별로 나눠진 특징점(visual word)들을 [n,128]형태로 모음, 모아진 특징점(visual word)들의 정보(비디오 내의 몇번째 프레임에서 나온 특징점인지)는 
# 같은 인덱스의 test_frame_total에서 확인 가능 및 비디오 내의 특정 프레임에서 나온 특징점(visual word)의 수는 test_local_info에서 확인 가능
test_frame_total = []
test_local_desc_total = []
test_local_info = {}
for k, v in test_local_desc.items():
    for kk, vv in v.items():
        l_num = 0
        if vv is not None:
            test_local_desc_total.extend(vv)
            test_frame_total.extend([k+", "+str(kk)] * len(vv))
            l_num = len(vv)
        test_local_info.update({k+", "+str(kk) : l_num})
test_local_desc_total = np.asarray(test_local_desc_total)
test_frame_total = np.asarray(test_frame_total)


print("\t{:3.2f}s\n\n".format(time.time()-start))



Aggregate SIFT descriptor
	4.78s




# 비디오 feature 기술을 위한 프레임 feature 기술 (Empty Module 2,3,4)

In [9]:
def clustering(train_desc, test_desc=None, n_clusters=200):
    #########################################################
    ## 모든 특징점들 중, 대표 특징점(codebook)을 선정하는 함수
    ## 
    ## Input 
    ##     train_desc : 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     test_desc : 모든 test 비디오의 모든 프레임에서 추출한 특징점(visual word)들
    ##     n_clusters : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     train_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     test_pred : 대표 특징점(codebook)에 대해 train_desc가 할당된 위치
    ##     clusters : 대표 특징점(codebook)
    ##     kmeans : kmeans 인스턴스
    #########################################################
    
    
    
    ##### Empty Module 2 : 대표 특징점(codebook) 선정 ######
    # 제약 조건 : MiniBatchKMeans 사용, random_state=0 고정
    # sklearn의 MiniBatchKMeans를 활용하여 n_clusters 크기 만큼의 대표 특징점(codebook)을 선정
    
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, random_state=0).fit(train_desc) # MiniBatchKmeans를 이용하여 train_desc를 학습 시킴
    # 함수의 반환값으로 kmeans 인스턴스가 있길래 MiniBatchMeans 모듈을 초기화하고, train 데이터를 학습시킨 형태를 keans변수에 저장하였습니다 !
    
    clusters = kmeans.cluster_centers_ # 대표 특징점(codebook)을 clusters 변수에 저장
    # cluster centers의 좌표를 반환하는 .cluster_centers_ Attributes를 이용하여 대표 특징점(codebook)을 지정
    
    ###########################################
    
    train_pred = kmeans.predict(train_desc)
    if test_desc is not None:
        test_pred = kmeans.predict(test_desc)
    else:
        test_pred = None
    return train_pred, test_pred, clusters, kmeans

In [10]:
# 모든 train 비디오의 모든 프레임에서 추출한 특징점(visual word)들로 대표 특징점(codebook)을 만들고,
# train 비디오의 모든 프레임에서 추출된 특징점(visual word)과 test 비디오의 모든 프레임에서 추출된 특징점(visual word)을 대표 특징점(codebook)에 할당
train_local_alloc, test_local_alloc, local_codebook, local_kmeans = clustering(train_local_desc_total, test_local_desc_total, args_local_cluster)

In [11]:
def VLAD(X, alloc, centers):
    #########################################################
    ## 이미지 feature인 VLAD feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     X : 한 프레임의 특징점(visual word)들
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     centers : 대표 특징점(codebook)
    ##
    ## Output
    ##     V : VLAD feature
    #########################################################
    
    m,d = X.shape
    k = centers.shape[0]
    
    # VLAD feature를 담기 위한 변수 - V : k x d size의 2차원 배열
    V=np.zeros([k,d])

    for i in range(k):
        if np.sum(alloc==i)>0:
            ####################### Empty Module 3 : VLAD ########################
            # 이미지에서 추출된 특징점(visual word) X와 이들이 대표 특징점(codebook)으로 할당된 정보 alloc을 이용해,
            # 동일한 대표 특징점(codebook)으로 할당된 특징점(visual word)들의 벡터 합 계산해 V[i]에 저장
            # hint : 바로 위 조건문의 조건 "alloc==i"의 의미를 파악하고 인덱싱에 활용
            
            V[i,:] = np.sum(X[i == alloc, :] - centers[i], axis=0) # V의 행에 동일한 대표 특징점으로 할당된 특징점들의 '벡터 합'을 저장
            
            # 프로젝트 개요 중 VLAD에 대한 설명에서 'VLAD는 각 이미지 내의 특징점(visual word)들을 대표 특징점(codebook)에 거리 순으로 할당해 둘 간의 "벡터 차이"를 계산한 후 
            # 동일한 대표 특징점(codebook)에 할당된 특징점(visual word)의 "벡터 차이 값"을 모두 더해주는 방식'이라는 말이 있었습니다. 
            # 따라서 한 프레임의 특징점들을 담고있는 X 중에서, 대표특징점의 인덱스(k)와 한 프레임의 특징점(X)들이 대표 특징점에 할당된 위치(alloc)가 같은 지점의 X값과 
            # 그 지점(인덱스)의 대표 특징점(즉, centers[i])를 뺀 값(즉, 벡터 차이 값)을 모두 더해주어야 합니다.
            # 벡터를 더하는 함수인 np.sum()에 X[i == alloc, :] - centers[i] 를 넣어서 코드를 작성하였습니다.
            # axis=0을 넣은 이유는 배열에서 한 프레임을 나타내는 열(x축)을 기준으로 함께 더해주기 위해서 입니다.
            
            ######################################################################
    
    # 후처리 과정
    V = V.flatten()
    V = np.sign(V)*np.sqrt(np.abs(V))
    if np.sqrt(np.dot(V,V))!=0:
        V = V/np.sqrt(np.dot(V,V))
    return V


def BoW(alloc, n_cluster):
    #########################################################
    ## 이미지 feature인 BoW feature를 기술하기 위한 함수
    ## 
    ## Input 
    ##     alloc : 한 프레임의 특징점(visual word)들이 대표 특징점(codebook)에 할당된 위치
    ##     n_cluster : 대표 특징점(codebook)의 수
    ##
    ## Output
    ##     V : BoW feature
    #########################################################
    
    ######################### Empty Module 4 : BoW ##########################
    # 이미지에서 추출된 특징점(visual word)이 대표 특징점(codebook)으로 할당된 정보 alloc의 histogram을 계산
    # np.histogram 함수 참고
    
    V = np.histogram(alloc, bins = range(n_cluster + 1), normed=True)[0]
    # histogram 함수 코드를 작성하기 위해 11주차 과제인 '2D 데이터 분류 문제'에서 [Empty Module #4] BoW: histogram vector 생성 부분을 참고하였습니다.
    # 함수의 파라미터 중 지정된 범위에서 동일한 너비 빈의 수를 정의하는 'bins'의 범위를 대표 특징점을 나타내는 n_cluster에서 +1 한 것을 범위로 지정하였습니다.
    
    #########################################################################
    return V

In [12]:
print("\n\nAllocate center & Descript local histogram")
start = time.time()

# Train 비디오 내의 프레임 별로 이미지 feature 기술 -> train_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> train_global_desc_key
train_global_desc = []
train_global_desc_key = []
vi=0
for k, v in train_local_info.items():
    if v!=0:
        if args_aggr=="bow":            
            hist_desc = BoW(train_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(train_local_desc_total[vi:vi+v], train_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        train_global_desc.append(hist_desc)
        train_global_desc_key.append(k)
train_global_desc = np.asarray(train_global_desc)
train_global_desc_key = np.asarray(train_global_desc_key)


# Test 비디오 내의 프레임 별로 이미지 feature 기술 -> test_global_desc
# 각 이미지 feature의 정보(속한 비디오 이름, 비디오 내의 인덱스) -> test_global_desc_key
test_global_desc = []
test_global_desc_key = []
vi=0
for k, v in test_local_info.items():
    if v!=0:
        if args_aggr=="bow":
            hist_desc = BoW(test_local_alloc[vi:vi+v], args_local_cluster)
        elif args_aggr=="vlad":
            hist_desc = VLAD(test_local_desc_total[vi:vi+v], test_local_alloc[vi:vi+v], local_codebook)
        else:
            import pdb; pdb.set_trace()

        vi+=v
        test_global_desc.append(hist_desc)
        test_global_desc_key.append(k)
test_global_desc = np.asarray(test_global_desc)
test_global_desc_key = np.asarray(test_global_desc_key)

print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript local histogram
	86.93s




In [13]:
# VLAD feature의 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_global_desc)
    train_global_desc = pca.transform(train_global_desc)
    test_global_desc = pca.transform(test_global_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))



Reduce dim of descriptor of the frames with PCA
	39.91s




In [14]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 train 비디오의 각 프레임 별 label 가공
train_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_global_desc_key])
train_global_label = []
for fid in train_global_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_global_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_global_label = np.asarray(train_global_label).ravel()

# 분류를 위해, 행동 분류에 대한 test 비디오의 각 프레임 별 id 가공 
test_global_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_global_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))


# train_global_desc : Train 비디오 내의 프레임 별로 이미지 feature 기술
# test_global_desc : Test 비디오 내의 프레임 별로 이미지 feature 기술
# trian_global_id : train data의 id
# train_global_label : train data의 label 값 (정답)
# test_global_id : test data의 id




Processing label
	0.90s




In [15]:
def saveFile(predict, predict_id, name, best_params=None):
    #########################################################
    ## 결과를 저장하기 위한 함수
    ## 
    ## Input 
    ##     predict : 모든 test 비디오의 행동 예측 값
    ##     predict_id : 모든 test 비디오의 id
    ##     name : 원하는 저장 파일 이름
    ##     best_params : 원하는 instance 저장 시 사용
    ##
    #########################################################
    
    data = np.concatenate((np.expand_dims(predict_id.astype("str"), axis=1), np.expand_dims(predict.astype("str"), axis=1)), axis=1)
    csv = pd.DataFrame(data, columns=['Id', 'Category'])
    csv.to_csv(name + ".csv", index=False)
    
    if best_params:
        f = open(name + ".pickle", "wb")
        pickle.dump(best_params, f, 2)

# 비디오의 모든 프레임에서 얻은 feature를 평균내어 비디오 feature 생성 및 분류 (Empty Module 5)

In [16]:
# grid search 사용하여 성능 높이기 << 시간이 너무 오래걸려서 주석처리 하였습니다!

# from sklearn.model_selection import GridSearchCV

# param = {'gamma' : ['auto', 'scale' ,1000, 100, 10, 1, 0.1, 0.001, 0.0001], 
#          'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000], 
#          'kernel' : ['linear', 'poly', 'rbf', 'sigmoid'],
#         'tol': [0.001, 0.01, 0.1, 1, 0.008]}
# 위 파라미터 조정했을 때
# grid search의 best_score_ : 0.26188~
# best parameter : {'C': 100, 'gamma': 1, 'tol' : 1} 
# 제출 성적 : 0.23762 ((별로!!!))

# 위 파라미터에서 C, gamma 파라미터만 조정했을 때
# grid search의 best_score_ : 0.2559405940594059
# best parameter: {'C': 10, 'gamma': 'scale'} 
# 제출 성적 : 0.26732

# grid search의 실행시간이 3384.99s로, 너무 오랜시간이 걸려서 C와 tol 파라미터를 실험적으로 바꾸며 학습시키고,
# train data의 score 값을 프린트 해보며 학습 데이터로 예측 점수를 보며 성능을 높이기로 했습니다

print("\n\nSVM global averaging in frame")
start = time.time()
#################### Empty Module 6 : SVM (averaging) ######################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 비디오 별로 평균 내어 비디오 feature로 사용
# 비디오 feature로 학습 후, 행동 예측
# hint : 위 셀에서 선언한 train_global_id, train_global_label, test_global_id 활용


# 우선, Train 비디오 내의 프레임 별로 이미지 feature 기술한 train_global_desc으로 학습을 시키고,
# Test 비디오 내의 프레임 별로 이미지 feature 기술한 test_global_desc으로 예측해야 합니다.

svm = SVC(random_state = 0, class_weight = 'balanced', C = 3, gamma = 4.78, tol = 0.5)

# gs = GridSearchCV(svm, param, scoring = 'accuracy', cv = 10, n_jobs = -1) # n_jobs : -1 means using all processors


### 각 프레임을 나타내는 이미지 featrue를 비디오 별로 '평균' 내기 ###
# 각 프레임을 나타내는 이미지 feature = 5개씩 -> for문을 5개씩 돌리면서 평균값 저장

# train data
train_x_mean = []
train_label_mean = [] # 평균값을 저장할 새로운 리스트 선언
for i in range(0, (train_global_id.shape)[0], 5): # 비디오 별로 '평균' 구해서 리스트에 추가 : np.mean()함수 활용 
    # - train_global_desc에 대한 id(=train_global_id)의 shape을 출력해보니 튜플 형태가 나왔습니다. 
    # 각 프레임을 나타내는 이미지 feature를 나누며 반복문을 돌리기 위해 튜플 중 첫번째 인텍스 값을 이용했습니다. : (train_global_id.shape)[0]
    train_x_mean.append(np.mean(train_global_desc[i:i+5, :], axis=0))
    train_label_mean.append(np.mean(train_global_label[i:i+5])) 
    
# test data
test_x_mean = [] # 평균값을 저장할 새로운 리스트 선언
for i in range(0, (test_global_id.shape)[0], 5): # 비디오 별로 '평균' 구해서 리스트에 추가 : np.mean()함수 활용
    # - test_global_desc에 대한 id(=test_global_id)의 shape을 출력해보니 튜플 형태가 나왔습니다. 
    # 각 프레임을 나타내는 이미지 feature를 나누며 반복문을 돌리기 위해 튜플 중 첫번째 인텍스 값을 이용했습니다. : (test_global_id.shape)[0]
    test_x_mean.append(np.mean(test_global_desc[i:i+5, :], axis=0))
    

# 모델을 학습시키기 위해 평균값을 넣은 리스트를 np.array 형태로 변환
train_x_mean = np.array(train_x_mean)
train_label_mean = np.array(train_label_mean).astype('int64') # 원래 label 값 : 정수형, but mean값은 float 형으로 저장 => int 형으로 바꿔주기
test_x_mean = np.array(test_x_mean)


# 모델 학습
svm.fit(train_x_mean, train_label_mean)
# print(gs.best_score_) # grid search에서 가장 좋은 성능
# print(gs.best_params_) # grid search에서 가장 좋은 성능을 보인 파라미터


# 실험적으로 파라미터를 조정할 때 학습 데이터에 대한 score을 찍어보며 대조하는데 사용
print(svm.score(train_x_mean, train_label_mean))
## C = 100 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.29504 
## C = 50 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.29504 
## C = 10 => 학습데이터에 대한 score : 1.0 
## C = 5 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.29504
# >> C값만을 계속해서 조정했을 때 학습데이터에 대한 score이 1.0이 나오는 것을 보며 과접합이 일어났을 수도 있겠다고 생각하였고,
# C값을 100, 50, 10, 5로 줄여가며 제출 성능을 비교해보았을 때 0.29504로 같은 것을 보며
# 다른 파라미터를 조정해보며 성능을 높여보아야 겠다고 판단하였습니다. 따라서 중지 기준에 대한 허용 오차인 tol 파라미터를 추가하여 조정해보았습니다.

## C = 5, tol = 0.1 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.29900
## C = 5, tol = 0.5 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.30693
## C = 5, tol = 0.7 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.29306
# >> C값은 고정시키고, tol 값을 좀 더 높여보니 학습데이터에 대한 score은 1.0으로 같지만, 제출 성능은 tol값이 0.1보다 0.5에서 높지만, 0.7로 더욱 높여보니 성능이 떨어짐을 알 수 있었습니다.
# 따라서 tol 값을 0.5로 고정시키고, C값을 좀 더 줄여가며 정규화 강도를 높여보고자 하였습니다.

## C = 2, tol = 0.5 => 학습데이터에 대한 score : 0.9886138613861386, 제출 성능 : 0.30891
## C = 2, tol = 0.1 => 학습데이터에 대한 score : 0.9896039603960396, 제출 성능 : 0.29702
# >> 이 부분을 보며 tol 값은 0.5가 적당하다는 것을 알 수 있었습니다.

## C = 3, tol = 0.5 => 학습데이터에 대한 score : 0.9985148514851485, 제출 성능 : 0.31485
## C = 4, tol = 0.5 => 학습데이터에 대한 score : 1.0, 제출 성능: 0.30693
## C = 3.5, tol = 0.5 => 학습데이터에 대한 score : 1.0, 제출 성능 : 0.30693
## C = 2.5, tol = 0.5 => 학습데이터에 대한 score : 0.9945544554455445, 제출 성능 : 0.30891
# >> tol값을 0.5로 고정하고, C값을 0.5 단위로 내려가며 줄이고 올려가며 실행해 본 결과 C = 3, tol = 0.5값이 가장 최고의 성능을 낸다는 것을 알 수 있었습니다.

# 더욱 성능을 향상하기 위해 C = 3, tol = 0.5을 고정하고 gamma값을 바꿔보았습니다.
# gamma = 'scale'이었을 때 최고 성능을 보인 것을 보고, gamma = scale(=1/(n_features(=128)*X.var())) = 4.762577136401144 를 구하고 그 근처의 값으로 gamma값을 바꿔보았습니다.
## gamma = 4.76 => 제출성능 : 0.31287
## gamma = 4.763 => 제출성능 : 0.31485
## gamma = 4.764 => 제출성능 : 0.31485
## gamma = 4.7627 => 제출성능 : 0.31485
## gamma = 4.767 => 제출성능 : 0.31485
## gamma = 4.77 => 제출성능 : 0.31683 => best
## gamma = 4.775 => 제출성능 : 0.31683 => best
## gamma = 4.78 => 제출성능 : 0.31683 => best
## gamma = 4.8 => 제출성능 : 0.31287

# best_model = gs.best_estimator_ # 최상의 성능을 내는 모델을 best_model에 저장

# test 데이터로 예측
svm_predict = svm.predict(test_x_mean)

###########################################################################
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_averaging")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global averaging in frame
0.9985148514851485
	2.38s




In [17]:
1/(128*np.var(train_x_mean)) # gamma = 'scale'값 계산

4.762577136401144

# 비디오의 모든 프레임에서 얻은 feature로 분류기를 사용해 예측하고 예측치 중 가장 많은 빈도를 나타낸 행동을 선정 (Empty Module 6)

In [18]:
# grid search를 통해 최적 모델 찾기 << 시간이 너무 오래걸려서 주석처리 하였습니다 !

# from sklearn.model_selection import GridSearchCV 
# param = {'gamma' : ['auto', 'scale' ,1000, 100, 10, 1, 0.1, 0.001, 0.0001], 
#          'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000, 3000]}
# C, gamma 파라미터만 조정했을 때
# grid search의 best_score_ : 0.21663366336633666
# best parameter : {'C': 1, 'gamma': 'scale'} 

# voting 부분에서는 grid search에 C, gamma 이외의 parameter를 더 추가하면 4시간 돌려봐도 결과값이 나오지 않았습니다..
# 또한, grid search을 통해 산출된 최적 모델로 결과를 제출해보았을 때 
# voting 방식이 0.22772, average 방식이 0.26732로, 상대적으로 voting 방식이 average 방식보다 성능이 낮게 나옴을 확인할 수 있었습니다
# 따라서 성능향상을 위해 파라미터를 계속해서 조정하는 과정은 voting에서 하지 않았습니다.

print("\n\nSVM global voting in frame")
start = time.time()
############################### Empty Module 5 : SVM (voting) ##################################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 각 프레임을 나타내는 이미지 feature를 모두 사용해 SVM 학습
# 학습 시, 각 이미지 feature의 label은 해당되는 비디오의 label(행동)로 사용
# 프레임 별로 행동 예측 후, 같은 비디오 내 프레임의 행동 예측 값의 최빈값을 해당 비디오의 행동 예측 값으로 선정
# hint : 위 셀에서 선언한 train_global_label, test_global_id 및 mode 활용

# svm = SVC(random_state = 0, class_weight = 'balanced') # 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# svm.fit(train_global_desc, train_global_label) # 각 프레임을 나타내는 이미지 feature를 모두 사용해 SVM 학습, 학습 시 각 이미지 feature의 label은 해당되는 비디오의 label(행동)로 사용
# svm_predict_original = svm.predict(test_global_desc) # 예측

# gs = GridSearchCV(svm, param, scoring = 'accuracy', cv = 10, n_jobs = -1) # 최적 모델을 찾기 위한 grid search 모듈 선언


# Train 비디오 내의 프레임 별로 이미지 feature 기술한 train_global_desc으로 학습을 시키고,
# Test 비디오 내의 프레임 별로 이미지 feature 기술한 test_global_desc으로 예측해야 합니다.

svm.fit(train_global_desc, train_global_label) # Empty module 5에서 선언한 svm 모듈을 이용해 학습데이터로 학습
# print(gs.best_score_)
# print(gs.best_params_)
# best_model = gs.best_estimator_ # 최적의 모델 저장


# test data에 대한 예측값 저장 
# - voting 방식은 행동 예측 값의 최빈값을 해당 비디오의 예측값을 선정해야하는 방식이기 때문에 행동 예측값을 'svm_predict_original'이라는 이름의 변수에 넣었습니다.
svm_predict_original = svm.predict(test_global_desc) 

### 프레임 별로 행동 예측 후, 같은 비디오 내 프레임의 행동 예측 값의 최빈값을 해당 비디오의 행동 예측 값으로 선정 ###
svm_predict = [] # 행동 예측값의 최빈값을 담을 새로운 list 생성
# 같은 비디오 내 각 프레임을 나타내는 이미지 feature = 5개씩 -> for문을 5개씩 돌리면서 최빈값 저장
for i in range(0, (test_global_id.shape)[0], 5):
    svm_predict.append(mode(svm_predict_original[i:i+5]).mode[0]) # 같은 비디오 내, 프레임의 행동 예측값의 최빈값을 새로운 리스트에 추가
## 이 모듈에서 사용하는 최빈값 함수 "from scipy.stats import mode" 는 mode(mode 값의 배열), count(각각의 mode에 대한 count 배열) 두 개의 배열을 반환합니다.
## svm_predict에 저장해야하는 값는 mode에 대한 값이지, mode에 대한 count를 알고 싶은 것이 아니기 때문에 ".mode[0]"으로 mode값의 배열 만을 넣어주어야 합니다.
    

# 최빈값을 담은 리스트를 np.array 형태로 변환
svm_predict = np.array(svm_predict) 
################################################################################################
saveFile(classinfo_arr[svm_predict][:,1], np.arange(len(test_list)), "svm_global_voting")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM global voting in frame
	15.07s




# 프레임 feature에서 대표되는 feature를 선정 후 BoW or VLAD 방식으로 비디오 feature를 기술 (Empty Module 7)

In [19]:
train_global_alloc, test_global_alloc, global_codebook, global_kmeans = clustering(train_global_desc, test_global_desc, args_global_cluster)

In [20]:
print("\n\nAllocate center & Descript global histogram")
start = time.time()
train_vid_names = np.asarray([i.split(", ")[0] for i in train_global_desc_key])
train_vid_names_u = np.unique(train_vid_names)

# Train 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
train_video_desc = []
train_video_desc_key = []
for vid_name in train_vid_names_u:
    cind = np.where(vid_name==train_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(train_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(train_global_desc[cind], train_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    train_video_desc.append(hist_desc)
    train_video_desc_key.append(vid_name)
train_video_desc = np.asarray(train_video_desc)
train_video_desc_key = np.asarray(train_video_desc_key)

# Test 비디오 내 프레임 별로 기술된 이미지 feature를 기반으로 한번 더 기술하여(한번 더 BoW 혹은 VLAD)
# 각 비디오에 대한 비디오 feature 기술
# Empty Module 7과 관련있으며, 5,6과는 무관
test_vid_names = np.asarray([i.split(", ")[0] for i in test_global_desc_key])
test_vid_names_u = np.unique(test_vid_names)

test_video_desc = []
test_video_desc_key = []
for vid_name in test_vid_names_u:
    cind = np.where(vid_name==test_vid_names)[0]
    if args_aggr=="bow":
        hist_desc = BoW(test_global_alloc[cind], args_global_cluster)
    elif args_aggr=="vlad":
        hist_desc = VLAD(test_global_desc[cind], test_global_alloc[cind], global_codebook)
    else:
        import pdb; pdb.set_trace()

    test_video_desc.append(hist_desc)
    test_video_desc_key.append(vid_name)
test_video_desc = np.asarray(test_video_desc)
test_video_desc_key = np.asarray(test_video_desc_key)


print("\t{:3.2f}s\n\n".format(time.time()-start))




Allocate center & Descript global histogram
	12.55s




In [21]:
print("\n\nProcessing label")
start = time.time()

# 분류를 위해, 행동 분류에 대한 각 train 비디오 별 label 가공
train_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in train_video_desc_key])
train_video_label = []
for fid in train_video_id:
    cind = np.where(train_csv_arr[:, 0]==fid)[0]
    clsname = train_csv_arr[cind, 1]
    cinfo_ind = np.where(classinfo_arr[:, 1] == clsname)[0]
    train_video_label.append(classinfo_arr[cinfo_ind, 0].astype("int"))
train_video_label = np.asarray(train_video_label).ravel()

# 분류를 위해, 행동 분류에 대한 각 test 비디오 별 id 가공
test_video_id = np.array([int(i.split("/")[-1].split(".")[0]) for i in test_video_desc_key])

print("\t{:3.2f}s\n\n".format(time.time()-start))


# train_video_desc, test_video_desc 각 비디오에 대한 비디오 feature 기술
# train_video_id : train data의 id 값
# train_video_label : train data의 label (정답)
# test_video_id : test data의 id 값



Processing label
	0.22s




In [22]:
# 이미지 feature에 대해 다시 한번 VLAD feature 기술 방식을 사용하여 video feature를 기술한 경우 큰 차원으로 인해 메모리 부족 현상이 발생하므로 PCA를 이용한 차원 축소
if args_aggr=="vlad":
    print("\n\nReduce dim of descriptor of the frames with PCA")
    start = time.time()
    pca = PCA(n_components=pca_vlad, random_state=0)
    pca.fit(train_video_desc)
    train_video_desc = pca.transform(train_video_desc)
    test_video_desc = pca.transform(test_video_desc)
    print("\t{:3.2f}s\n\n".format(time.time()-start))




Reduce dim of descriptor of the frames with PCA
	8.63s




In [23]:
# 최적의 모델을 선정하기 위해 grid search를 사용 << 시간이 많이 들어 주석처리

# from sklearn.model_selection import GridSearchCV

# param = {'gamma' : ['auto', 'scale' ,1000, 100, 10, 1, 0.1, 0.001, 0.0001], 
#          'C' : [0.001, 0.01, 0.1, 1, 10, 100, 1000]}
# 위의 파라미터만 조정했을 때
# grid search의 best_score_ : 0.12920792079207918
# best parameter : {'C': 1, 'gamma': 'scale'}

# grid search을 통해 산출된 최적 모델로 결과를 제출해보았을 때 
# voting 방식이 0.22772, average 방식이 0.26732, 비디오 feature방식이 0.14851로, 가장 성능이 낮게 나옴을 확인할 수 있었습니다.
# 따라서 성능향상을 위해 파라미터를 계속해서 조정하는 과정은 비디오 feature 방식에서 하지 않았습니다.

print("\n\nSVM video descriptor")
start = time.time()
######################## Empty Module 7 : SVM (video feature) ##########################
# 제약조건 : sklearn의 SVC 사용 및 random_state=0으로 고정
# 이전 이미지 feature를 기반으로 각 기술방식(BoW, VLAD)을 한번 더 적용해 만든 비디오 feature 사용
# 비디오 feature로 학습 후, 행동 예측

# grid search를 이용하여 최적의 모델 찾기
# gs = GridSearchCV(svm, param, scoring = 'accuracy', cv = 10, n_jobs = -1)

# 각 비디오에 대한 비디오 feature 기술한 train_video_desc와 train_videio_label으로 학습을 시키고, test_video_desc으로 예측해야 합니다.

# 학습 : 위 모듈에서 사용한 svm으로 train data를 학습 시켰습니다.
svm.fit(train_video_desc, train_video_label)

# grid search 결과를 확인해보기 위해 출력
# print(gs.best_score_)
# print(gs.best_params_)
# best_model = gs.best_estimator_ # 최적의 모델 저장


svm_predict = svm.predict(test_video_desc) # 예측값 저장
#######################################################################################
saveFile(classinfo_arr[svm_predict][:,1], test_video_id, "svm_video")
print("\t{:3.2f}s\n\n".format(time.time()-start))



SVM video descriptor
	1.13s


